<a href="https://colab.research.google.com/github/phibro36/DataScience-Tools-Ecosystem/blob/main/Stock_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
import os

# Updated list of stock tickers from the provided list
tickers = [
    "SOFI", "DRUG", "SRRK", "FOXO", "UXIN", "ARBB", "FREY", "ATGL", "STEC", "TWG", "FXLV",
    "JDZG", "AUST", "LSB", "CAPR", "AMLX", "ELWS", "CMND", "MHUA", "FNMA", "TVGN", "EGRX",
    "JDST", "NVOS", "DUST", "BLDP", "NUGT", "FMCC", "TSBX", "AMD", "F", "COTY", "MGM", "DNMR"
]

# Fetch data for the last 180 days
end_date = datetime.today().strftime('%Y-%m-%d')
start_date = (datetime.today() - timedelta(days=365)).strftime('%Y-%m-%d')  # 1 year for 52-week high calculation

def fetch_stock_data(tickers, start, end):
    all_data = []
    for ticker in tickers:
        stock = yf.Ticker(ticker)
        hist = stock.history(start=start, end=end)
        hist['Ticker'] = ticker  # Add a column to indicate which stock the data belongs to
        hist.reset_index(inplace=True)  # Reset index to make Date a column

        # Convert any datetime columns to timezone-unaware
        if 'Date' in hist.columns:
            hist['Date'] = pd.to_datetime(hist['Date']).dt.tz_localize(None)

        # Calculate 52-week high
        hist['52_Week_High'] = hist['High'].rolling(window=252, min_periods=1).max()  # 252 trading days in a year

        # Calculate 50-day moving average
        hist['50_Day_Moving_Avg'] = hist['Close'].rolling(window=50, min_periods=1).mean()

        all_data.append(hist)
    return pd.concat(all_data, axis=0)  # Combine data for all tickers into one DataFrame

stock_data = fetch_stock_data(tickers, start_date, end_date)

# Specify the directory where you want to save the file
output_directory = "C:\\Users\\phbrooks\\Desktop"  # Change this to your desired directory
os.makedirs(output_directory, exist_ok=True)  # Create directory if it doesn't exist
output_filename = os.path.join(output_directory, 'stock_data.xlsx')

# Export data to the specified directory
stock_data.to_excel(output_filename, index=False)

print(f"Data has been exported to {output_filename}")

# Optionally, reset display options (if you modified them)
pd.reset_option('display.max_rows')


Data has been exported to C:\Users\phbrooks\Desktop/stock_data.xlsx


In [23]:
import yfinance as yf
import pandas as pd
from textblob import TextBlob

# Updated Stock symbols and company names with all 34 stocks
stock_data = {
    "Stock Symbol": [
        "SOFI", "JDZG", "STEC", "LSB", "TWG", "DRUG", "DNMR", "MHUA", "AUST", "TVGN",
        "UXIN", "FREY", "NVOS", "SRRK", "ELWS", "TSBX", "ARBB", "AMLX", "CAPR", "BLDP",
        "JDST", "FNMA", "DUST", "EGRX", "ATGL", "CMND", "NUGT", "FMCC", "FOXO", "F",
        "AMD", "MGM", "FXLV", "COTY"
    ],
    "Company Name": [
        "SoFi Technologies, Inc.", "JIADE Limited", "Santech Holdings Limited",
        "LakeShore Biopharma Co., Ltd", "Top Wealth Group Holding Limited",
        "Bright Minds Biosciences Inc", "Danimer Scientific, Inc.",
        "Meihua International Medical Technologies", "Austin Gold Corp.",
        "Tevogen Bio Holdings Inc.", "Uxin Limited", "FREYR Battery",
        "Novo Integrated Sciences", "Scholar Rock Holding Corporation",
        "Earlyworks", "Turnstone Biologics", "ARB IOT Group",
        "Amylyx Pharmaceuticals", "Capricor Therapeutics, Inc",
        "Ballard Power Systems Inc.", "Direxion Daily Jr Gld Mnrs Bear 3X Shrs",
        "Federal National Mortgage Association", "Direxion Daily Gold Miners ETF-DUST",
        "Eagle Pharmaceuticals", "Alpha Technology Group Limited",
        "Clearmind Medicine Inc.", "Direxion Daily Gold Miners ETF-NUGT",
        "Federal Home Loan Mortgage Corporation", "FOXO Technologies Inc.",
        "Ford Motor Company", "Advanced Micro Device",
        "MGM Resorts International", "F45 Training", "Coty Inc."
    ]
}

# Fetch stock data and calculate scores
def fetch_and_rank_stocks():
    stock_details = []
    for symbol, name in zip(stock_data["Stock Symbol"], stock_data["Company Name"]):
        try:
            # Fetch stock data from Yahoo Finance
            stock = yf.Ticker(symbol)
            history = stock.history(period="3mo")  # Fetch 3-month price history
            info = stock.info

            # Extract financial metrics
            pe_ratio = info.get("trailingPE", 0)
            eps = info.get("trailingEps", 0)
            market_cap = info.get("marketCap", 0)
            day_low = info.get("dayLow", 0)
            day_high = info.get("dayHigh", 0)
            current_price = info.get("regularMarketPrice", 0)

            # Calculate 3-month percentage change
            if not history.empty:
                price_3mo_ago = history.iloc[0]['Close']
                performance_3mo = ((current_price - price_3mo_ago) / price_3mo_ago) * 100
            else:
                performance_3mo = 0

            # Calculate volatility (standard deviation of daily returns)
            history['Daily Return'] = history['Close'].pct_change()
            volatility = history['Daily Return'].std() * 100 if not history.empty else 0

            # Combine performance and volatility into a single score
            performance_score = (performance_3mo * 0.6) + (volatility * 0.4)

            # Calculate scores
            fundamental_score = (pe_ratio + eps + (market_cap / 1e9)) / 3 if pe_ratio and eps and market_cap else 0
            technical_score = ((day_high - day_low) / day_low * 100) if day_low and day_high else 0
            sentiment_score = calculate_sentiment_score()

            # Weighted composite score (higher weight for performance score)
            composite_score = (
                0.2 * fundamental_score +
                0.2 * technical_score +
                0.4 * performance_score +
                0.2 * sentiment_score
            )

            # Append stock details
            stock_details.append({
                "Stock Symbol": symbol,
                "Company Name": name,
                "Fundamental Score": round(fundamental_score, 2),
                "Technical Score": round(technical_score, 2),
                "3-Month Performance Score": round(performance_score, 2),
                "Sentiment Score": round(sentiment_score, 2),
                "Composite Score": round(composite_score, 2),
                "Notes on Ranking": f"PE: {pe_ratio}, EPS: {eps}, Market Cap: {market_cap}, Volatility: {round(volatility, 2)}%"
            })

        except Exception as e:
            stock_details.append({
                "Stock Symbol": symbol,
                "Company Name": name,
                "Fundamental Score": 0,
                "Technical Score": 0,
                "3-Month Performance Score": 0,
                "Sentiment Score": 0,
                "Composite Score": 0,
                "Notes on Ranking": f"Error: {str(e)}"
            })

    # Convert to DataFrame
    data = pd.DataFrame(stock_details)

    # Calculate overall scores and rank
    data["Rank"] = data["Composite Score"].rank(ascending=False).astype(int)

    # Sort by rank
    data = data.sort_values("Rank")
    return data

# Perform sentiment analysis (stub function, returns neutral sentiment)
def calculate_sentiment_score():
    # Default neutral sentiment
    return 5

# Save ranked stocks to a local Excel file
def save_to_local_drive(ranked_stocks, file_path="Ranked_Stocks_with_Insights.xlsx"):
    ranked_stocks.to_excel(file_path, index=False)
    print(f"Ranked stocks saved to {file_path}")

# Main function
def main():
    ranked_stocks = fetch_and_rank_stocks()
    save_to_local_drive(ranked_stocks)

# Execute script
if __name__ == "__main__":
    main()


Ranked stocks saved to Ranked_Stocks_with_Insights.xlsx
